## To Keras Tensorflow

In [ ]:
%pip install tensorflow transformers nobuco

# required install torch on https://pytorch.org/get-started/locally/

In [ ]:
import nobuco
from nobuco import ChannelOrder, ChannelOrderingStrategy
import torch
import torch.nn.functional as F
import tensorflow as tf
from model import Mamba, ModelArgs
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')

In [2]:
@nobuco.converter(F.softplus, channel_ordering_strategy=ChannelOrderingStrategy.MINIMUM_TRANSPOSITIONS)
def softplus(input: torch.Tensor):
    return lambda input: tf.keras.activations.softplus(input)

In [3]:
@nobuco.converter(torch.einsum, channel_ordering_strategy=ChannelOrderingStrategy.FORCE_PYTORCH_ORDER)
def converter_einsum(*args):
    def func(*args):
        equation = args[0]
        operands = args[1:]
        return tf.einsum(equation, *operands)
    return func

In [4]:
args = ModelArgs(
    d_model=5,
    n_layer=1,
    vocab_size=50277
)
model = Mamba(args)
model.eval()
export_name = "mamba_minimal_1_layer"
dummy_input = "Test"
input_ids = tokenizer(dummy_input, return_tensors='pt').input_ids

keras_model = nobuco.pytorch_to_keras(
    model,
    args=[input_ids], kwargs=None,
    input_shapes={input_ids: (1, None)}, # Annotate dynamic axes with None
    inputs_channel_order=ChannelOrder.TENSORFLOW,
    outputs_channel_order=ChannelOrder.TENSORFLOW,
    constants_to_variables=False,
    trace_shape=True,
    save_trace_html=True
)

2024-02-19 19:47:13.408601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-19 19:47:13.409221: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-02-19 19:47:13.481166: I external/local_xla/xla/service/service.cc:168] XLA service 0xea8a220 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-02-19 19:47:13.481192: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1708368433.488287   28138 device_compiler.h:18

Legend:
    Green — conversion successful
    Yellow — conversion imprecise
    Red — conversion failed
    Red — no converter found
    Bold — conversion applied directly
    * — subgraph reused
    Tensor — this output is not dependent on any of subgraph's input tensors
    Tensor — this input is a parameter / constant
    Tensor — this tensor is useless

Mamba[model](int64_0<1,1>) -> float32_87<1,1,50280>
 │  Embedding[torch.nn.modules.sparse](int64_0<1,1>) -> float32_2<1,1,5>
 │   └· embedding[torch.nn.functional](int64_0<1,1>, float32_1<50280,5>, None, None, 2.0, False, False) -> float32_2<1,1,5>
 │  ResidualBlock[model](float32_2<1,1,5>) -> float32_79<1,1,5>
 │   │  RMSNorm[model](float32_2<1,1,5>) -> float32_9<1,1,5>
 │   │   │  pow[torch.Tensor](float32_2<1,1,5>, 2) -> float32_3<1,1,5>
 │   │   │  mean[torch.Tensor](float32_3<1,1,5>, -1, keepdim=True) -> float32_4<1,1,1>
 │   │   │  __add__[torch.Tensor](float32_4<1,1,1>, 1e-05) -> float32_5<1,1,1>
 │   │   │  rsqrt[torch](floa

In [ ]:
tf.keras.models.save_model(keras_model, f'{export_name}.keras')
tf.keras.models.save_model(keras_model, f'{export_name}.h5')
tf.saved_model.save(keras_model, f'{export_name}')

In [7]:
# Inference Test for nobuco converted model
# prompt
dummy_prompt_keras = "Harry Potter test"
input_ids_keras = tokenizer(dummy_prompt_keras, return_tensors='tf').input_ids

#input_ids_keras = tf.cast(input_ids_keras, tf.int64)
# inference
out = keras_model.predict(input_ids_keras)
# output
print(out)

1/1 [==============================] - 0s 12ms/step
[[[-3.0078745   3.9671001  -2.305275  ]
  [-1.5027751  -1.864156    2.8154929 ]
  [-2.2186959   5.2344885  -1.0442678 ]
  ...
  [-4.8536053   5.0152903  -3.8399842 ]
  [-0.11023474  0.26254967  2.5834856 ]
  [ 1.321992   -1.8405054  -1.6095277 ]]]


## SavedModel 

In [7]:
# prompt
dummy_prompt_keras = "Harry Potter"
input_ids_keras = tokenizer(dummy_prompt_keras, return_tensors='tf').input_ids

# loading model
export_path = "mamba_minimal_1_layer" # No '.h5' in the path!
keras_model_restored = tf.saved_model.load(export_path)

input_ids_keras = tf.cast(input_ids_keras, tf.int64)

# inference
out = keras_model_restored(input_ids_keras)

# output
print(out)

tf.Tensor(
[[[ 2.1814482   5.0091105 ]
  [-0.9071193  -2.4775052 ]
  [-2.2117083  -0.39902794]
  ...
  [-0.49433738  1.2557342 ]
  [-2.74676    -0.9496337 ]
  [ 1.0100164  -4.3799496 ]]], shape=(1, 50280, 2), dtype=float32)


## .keras

In [10]:
# prompt
dummy_prompt_keras = "Harry Potter"
input_ids_keras = tokenizer(dummy_prompt_keras, return_tensors='tf').input_ids

# loading model
export_name = "mamba_minimal_1_layer"
keras_model = tf.keras.saving.load_model (f'{export_name}.keras', safe_mode=False)

input_ids_keras = tf.cast(input_ids_keras, tf.int64)

# inference
out = keras_model.predict(input_ids_keras)

# output
print(out)

AttributeError: Exception encountered when calling layer "lambda_12" (type Lambda).

'list' object has no attribute 'shape'

Call arguments received by layer "lambda_12" (type Lambda):
  • inputs=['tf.Tensor(shape=(1, None, 10), dtype=float32)', [['tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)'], ['tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=f

## .h5

In [11]:
# prompt
dummy_prompt_keras = "Harry Potter"
input_ids_keras = tokenizer(dummy_prompt_keras, return_tensors='tf').input_ids

# loading model
export_name = "mamba_minimal_1_layer"
keras_model = tf.keras.saving.load_model (f'{export_name}.h5')

input_ids_keras = tf.cast(input_ids_keras, tf.int64)

# inference
out = keras_model.predict(input_ids_keras)

# output
print(out)

AttributeError: Exception encountered when calling layer "lambda_12" (type Lambda).

'list' object has no attribute 'shape'

Call arguments received by layer "lambda_12" (type Lambda):
  • inputs=['tf.Tensor(shape=(1, None, 10), dtype=float32)', [['tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)'], ['tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=float32)', 'tf.Tensor(shape=(), dtype=f